# Logistics Regression Application to Predict Absenteeism

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

# Importing Preprocessed Dataset

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


# Targets for Absenteeism

In [4]:
#Creating targets
#Anyone above 3 hr absent: excessively absent, otherwise moderately absent

targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median() ,1,0)

In [5]:
data_preprocessed['Excessive Absenteeism'] = targets
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [6]:
#Ratio of 1's:
targets.sum() / len(targets)

0.45571428571428574

In [7]:
#Drop insignificant features - after backward elimination
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours', 'Day of the Week', 'Daily Work Load Average','Distance to Work'], axis = 1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [8]:
data_with_targets.shape

(700, 12)

# Scaling Dataset

In [9]:
#Assigning columns except the target column to a new variable, that is unscaled dataset
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [10]:
#Prebuilt Standard and Custom scaler - taken from instructor

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]
        

In [11]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [12]:
#The columns below are omitted because they are binary and standardization of them would lose its effect.     
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [13]:
#Scale everything except columns to omit defined above
columns_to_scale = [x for x in  unscaled_inputs.columns.values if x not in columns_to_omit]

In [14]:
#Scale 'columns_to_scale' array with prebuilt Custom Scaler
absenteeism_scaler = CustomScaler(columns_to_scale)

In [15]:
#To store mean and std in absenteeism_scaler object
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [47]:
#Create scaled_inputs dataframe from absenteeism scaler given unscaled_inputs
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [17]:
scaled_inputs.shape

(700, 11)

# Splitting Data into Train and Test Sets

In [18]:
#Import splitter, determine shuffler and training size

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [19]:
print(x_train.shape,y_train.shape)

(560, 11) (560,)


In [20]:
print(x_test.shape,y_test.shape)

(140, 11) (140,)


# Logistics Regression Application

In [21]:
#Import logisticRegression 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Model Training

In [22]:
reg = LogisticRegression()

In [23]:
#Reg object to be filled
reg.fit(x_train,y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
reg.score(x_train,y_train)  #Accuracy around %80 is reached.

0.775

Manually Checking Model Accuracy

In [52]:
model_outputs = reg.predict(x_train)

In [27]:
#Manually checked % accuracy
np.sum(model_outputs == y_train) / len(model_outputs)  #Manually calculated accuracy

0.775

Intercepts & Coefficients

In [28]:
#Intercept i.e. Bias
reg.intercept_

array([-1.46547112])

In [29]:
#Coefficients i.e. weights
reg.coef_

array([[ 2.62749942,  0.86338637,  2.96050661,  0.66390745,  0.15493732,
         0.59979822, -0.17245127,  0.27568526, -0.23452541,  0.34249662,
        -0.2775137 ]])

In [30]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [31]:
feature_name = unscaled_inputs.columns.values

In [32]:
summary_table = pd.DataFrame (columns=['Feature Name'],data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature Name,Coefficient
0,Reason_1,2.627499
1,Reason_2,0.863386
2,Reason_3,2.960507
3,Reason_4,0.663907
4,Month Value,0.154937
5,Transportation Expense,0.599798
6,Age,-0.172451
7,Body Mass Index,0.275685
8,Education,-0.234525
9,Children,0.342497


In [33]:
#Coefficient i.e. weights table
summary_table.index = summary_table.index +1 
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.465471
1,Reason_1,2.627499
2,Reason_2,0.863386
3,Reason_3,2.960507
4,Reason_4,0.663907
5,Month Value,0.154937
6,Transportation Expense,0.599798
7,Age,-0.172451
8,Body Mass Index,0.275685
9,Education,-0.234525


In [34]:
#Adding odds ratio, to see marginal effect of each feature
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)

summary_table.sort_values('Odds Ratio', ascending = False)
#If coefficient is around 0 OR Odds ratio is around 1, a feature is not quite important.

,Feature Name,Coefficient,Odds Ratio
3,Reason_3,2.960507,19.307751
1,Reason_1,2.627499,13.839121
2,Reason_2,0.863386,2.371177
4,Reason_4,0.663907,1.942367
6,Transportation Expense,0.599798,1.821751
10,Children,0.342497,1.408460
8,Body Mass Index,0.275685,1.317433
5,Month Value,0.154937,1.167585
7,Age,-0.172451,0.841599
9,Education,-0.234525,0.790946


In [35]:
#Backward Elimination is When we have p values, get rid of all features having < 0.05, or odds ratio having values close to 1, 
#or coeff. around 0 --> remove insignificant features and re-run code.

# Testing the Model

In [36]:
reg.score(x_test,y_test)

0.75

In [54]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba[:10]

array([[0.71221976, 0.28778024],
       [0.58760009, 0.41239991],
       [0.44337438, 0.55662562],
       [0.77903962, 0.22096038],
       [0.08458343, 0.91541657],
       [0.33103371, 0.66896629],
       [0.29792496, 0.70207504],
       [0.12956221, 0.87043779],
       [0.78307821, 0.21692179],
       [0.74708659, 0.25291341]])

In [42]:
np.mean(predicted_proba[:,1])

0.47065683116119095

In [55]:
predicted_proba[:10]

array([[0.71221976, 0.28778024],
       [0.58760009, 0.41239991],
       [0.44337438, 0.55662562],
       [0.77903962, 0.22096038],
       [0.08458343, 0.91541657],
       [0.33103371, 0.66896629],
       [0.29792496, 0.70207504],
       [0.12956221, 0.87043779],
       [0.78307821, 0.21692179],
       [0.74708659, 0.25291341]])

Final Model

In [49]:
import pickle

In [50]:
with open('model', 'wb') as file:
    pickle.dump(reg,file)
    
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)